<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/CIS%20eFAS%20Industry%20DB%20Scraping/IT_EFAS_INDU_COMMON_DB/IT_EFAS_INDU_COMMON_DB_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##이 부분은 처음 한번만 실행함

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.B6sinVrbiT/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.UavsQqM7np/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.CGgocYAsKb/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [2]:
import pandas as pd

# initialize list of lists
mappingData = [['1', '농림어업', ''],
['2', '광업', ''],
['3', '음식료', '(1401)음식료'],
['4', '섬유', '(1403)섬유'],
['5', '의류', '(1404)의류'],
['6', '가죽신발', '(1405)가죽·신발'],
['7', '목재', '(1406)목재'],
['8', '제지', '(1407)제지'],
['9', '인쇄', '(1408)인쇄'],
['10' '기타 제조업', '(1410)기타 제조업'],
['11', '석유정제', '(1301)석유정제'],
['12', '석유화학', '(1201)석유화학'],
['13', '정밀화학', '(1202)정밀화학'],
['14', '의약', '(1101)의약'],
['15', '타이어', ''],
['16' '(1302)고무'],
['17', '플라스틱', '(1303)플라스틱'],
['18', '유리', '(1304)유리'],
['19', '세라믹', '(1305)세라믹'],
['20', '시멘트', '(1306)시멘트'],
['21', '기타 비금속 광물', '(1307)기타 비금속 광물'],
['22', '철강', '(1308)철강'],
['23', '비철금속', '(1309)비철금속'],
['24', '주조', '(1310)주조'],
['25', '조립금속', '(1311)조립금속'],
['26', '반도체', '(1102)반도체'],
['27', '디스플레이', '(1103)디스플레이'],
['28', '기타 전자부품', ''],
['29', '컴퓨터', '(1104)컴퓨터'],
['30', '통신기기', '(1105)통신기기'],
['31', '가전', '(1106)가전'],
['32', '정밀기기', '(1107)정밀기기'],
['33', '전기기기', '(1204)전기기기'],
['34', '전지', '(1108)전지'],
['35', '일반목적기계', '(1205)일반목적기계'],
['36', '특수목적기계', '(1206)특수목적기계'],
['37', '자동차', '(1207)자동차'],
['38', '자동차부품', ''],
['39', '조선', '(1312)조선'],
['40', '기타 수송장비', '(1209)기타 수송장비'],
['41', '항공', '(1109)항공'],
['42', '가구', '(1409)가구'],
['43', '전기가스수도', ''],
['44', '폐수처리및자원재활용', ''],
['45', '건설', ''],
['46', '도소매', '(2101)도·소매업'],
['47', '육상창고', ''],
['48', '해운', ''],
['49', '항공운송', ''],
['50', '숙박·음식점', '(2401)숙박·음식점'],
['51', '출판', '(2201)출판'],
['52', '방송', '(2202)방송'],
['53', '통신', '(2203)통신'],
['54', '정보', '(2204)정보'],
['55', '금융·보험', '(2205)금융·보험'],
['56', '부동산', '(2206)부동산'],
['57', '전문·과학기술', '(2208)전문·과학기술'],
['58', '사업시설관리서비스', '(2209)사업시설관리서비스'],
['59', '사업지원', '(2210)사업지원'],
['60', '임대', '(2207)임대'],
['61', '공공행정', ''],
['62', '교육', '(2303)교육'],
['63', '의료·보건', '(2304)의료·보건'],
['64', '사회복지', '(2305)사회복지'],
['65', '예술·스포츠·여가', '(2402)예술·스포츠·여가'],
['66', '기타 서비스','(2403)기타 서비스']]
 
# Create the pandas DataFrame
dfEFAStoISTANSMAP = pd.DataFrame(mappingData, columns=['EFAS_CD', 'EFAS_NM', 'ISTANS_NM'])
# print dataframe.
dfEFAStoISTANSMAP

,EFAS_CD,EFAS_NM,ISTANS_NM
0,1,농림어업,
1,2,광업,
2,3,음식료,(1401)음식료
3,4,섬유,(1403)섬유
4,5,의류,(1404)의류
...,...,...,...
61,62,교육,(2303)교육
62,63,의료·보건,(2304)의료·보건
63,64,사회복지,(2305)사회복지
64,65,예술·스포츠·여가,(2402)예술·스포츠·여가


In [10]:
import time
from datetime import date
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

#Since Colab does not make web browser by default, set the web browser 
options = Options()
options.add_argument('--headless')        # Set Head-less
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.headless = True

def iSTANSCall(istansUrl):
  browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)
  browser.get(istansUrl)  # Get data from istansUrl
  time.sleep(20) 
  # Move to inner iframe to set query condition ("newSuTabMain > searchFrame")
  browser.switch_to.frame("newSuTabMain")
  browser.switch_to.frame("searchFrame")
  # Set query condition : From JAN 2010
  select = Select(browser.find_element(By.XPATH, '//*[@id="df_period"]'))
  print(select.first_selected_option.get_attribute("value"))
  select.select_by_visible_text('월')
  select.select_by_value('4')
  time.sleep(10)
  select = Select(browser.find_element(By.XPATH, '//*[@id="from_year_month"]'))
  select.select_by_visible_text('2010')
  select.select_by_value('2010')
  time.sleep(10)
  select = Select(browser.find_element(By.XPATH, '//*[@id="from_month"]'))
  select.select_by_visible_text('1')
  select.select_by_value('1')
  time.sleep(10)
  browser.find_element(By.XPATH, '//*[@id="resultDiv"]/div[1]/div[1]/a').send_keys(Keys.ENTER)  # Click 'Query' button
  print("진행중_1")
  time.sleep(30)
  # Get HTML page source (Append Header, Body, and induName)
  dfHeader = pd.read_html(browser.page_source)[2]
  dfBody = pd.read_html(browser.page_source)[4]
  induName = pd.read_html(browser.page_source)[3]
  dfBody.iloc[:,0] = induName.iloc[:,[1]].values
  dfIStansResult = dfHeader.append(dfBody)
  print("진행중_2")
  # Refine Dataframe
  dfIStansResult.reset_index(drop=False)
  dfIStansResult.iloc[0, 0] = 'iStansInduNM'
  dfIStansResult.columns = dfIStansResult.iloc[0]
  dfIStansResult = dfIStansResult[1:]
  # Chage data to long type using 'melt' function
  dfIStansResult = pd.melt(dfIStansResult, id_vars=['iStansInduNM'], value_name='indexValue').sort_values('iStansInduNM')
  dfIStansResult.rename(columns={0 : 'STD_YM'}, inplace = True)
  dfIStansResult['STD_YM'] = dfIStansResult['STD_YM'].str.replace('년 ', '')
  dfIStansResult['STD_YM'] = dfIStansResult['STD_YM'].str.replace('월', '')
  dfIStansResult = dfIStansResult.reset_index().merge(dfEFAStoISTANSMAP, left_on='iStansInduNM', right_on='ISTANS_NM', how="inner")       
  # Move browser to main Frame
  browser.switch_to.default_content()
  WebDriverWait(browser, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID,"newSuTabMain")))
  browser.close()
  return dfIStansResult

<ipython-input-10-836ed3f02736>:17: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [11]:
# 생산지수(원지수) : Istans > 주제별 통계 > 산업동향지수 > 생산지수(원지수)
url = "https://istans.or.kr/su/newSuTab.do?scode=S120" 
iSTANSCallResult = iSTANSCall(url)
dfPRODUC_INDEX = iSTANSCallResult.rename(columns = {'indexValue':'PRODUC_INDEX'})

# 서비스업생산지수(원지수, 불변지수) : Istans > 주제별 통계 > 산업동향지수 > 서비스업생산지수(불변지수)
url = "https://istans.or.kr/su/newSuTab.do?scode=S358" 
iSTANSCallResult = iSTANSCall(url)
dfServPRODUC_INDEX = iSTANSCallResult.rename(columns = {'indexValue':'PRODUC_INDEX'})

dfPRODUC_INDEX = dfPRODUC_INDEX.append(dfServPRODUC_INDEX)
dfPRODUC_INDEX = dfPRODUC_INDEX[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'PRODUC_INDEX']]
dfPRODUC_INDEX.drop_duplicates(inplace=True)
display(dfPRODUC_INDEX)

<ipython-input-10-836ed3f02736>:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)


1
aaaaa
dadf
1
aaaaa
dadf


,STD_YM,EFAS_CD,EFAS_NM,PRODUC_INDEX
0,202202,14,의약,139.2
1,201304,14,의약,100.19
2,201505,14,의약,93.1
3,201509,14,의약,100.9
4,201208,14,의약,91.0
...,...,...,...,...
5258,201801,66,기타 서비스,97.68
5260,201802,66,기타 서비스,88.84
5262,201803,66,기타 서비스,104.6
5264,201804,66,기타 서비스,101.36


In [12]:
# 생산지수(계절조정) : Istans > 주제별 통계 > 산업동향지수 > 생산지수(계절조정)
url = "https://istans.or.kr/su/newSuTab.do?scode=S350" 
iSTANSCallResult = iSTANSCall(url)
dfPRODUC_INDEX_GAE = iSTANSCallResult.rename(columns = {'indexValue':'PRODUC_INDEX_GAE'})

# 서비스업생산지수(계절조정) : Istans > 주제별 통계 > 산업동향지수 > 생산지수(계절조정)
url = "https://istans.or.kr/su/newSuTab.do?scode=S359" 
iSTANSCallResult = iSTANSCall(url)
dfServPRODUC_INDEX_GAE = iSTANSCallResult.rename(columns = {'indexValue':'PRODUC_INDEX_GAE'})

dfPRODUC_INDEX_GAE = dfPRODUC_INDEX_GAE.append(dfServPRODUC_INDEX_GAE)
dfPRODUC_INDEX_GAE = dfPRODUC_INDEX_GAE[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'PRODUC_INDEX_GAE']]
dfPRODUC_INDEX_GAE.drop_duplicates(inplace=True)
display(dfPRODUC_INDEX_GAE)

<ipython-input-10-836ed3f02736>:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)


4
aaaaa
dadf
4
aaaaa
dadf


,STD_YM,EFAS_CD,EFAS_NM,PRODUC_INDEX_GAE
0,202202,14,의약,154.4
1,201304,14,의약,97.53
2,201505,14,의약,96.1
3,201509,14,의약,99.8
4,201208,14,의약,98.37
...,...,...,...,...
5258,201801,66,기타 서비스,100.49
5260,201802,66,기타 서비스,101.35
5262,201803,66,기타 서비스,102.51
5264,201804,66,기타 서비스,100.8


In [ ]:
dfPRODUC_INDEX_GAE

,STD_YM,EFAS_CD,EFAS_NM,PRODUC_INDEX_GAE
0,201708,14,의약,122.4
1,201707,14,의약,121.3
2,201101,14,의약,98.53
3,202107,14,의약,150.6
4,201010,14,의약,94.21
...,...,...,...,...
5223,201712,66,기타 서비스,101.38
5225,201801,66,기타 서비스,100.49
5227,201802,66,기타 서비스,101.35
5229,201803,66,기타 서비스,102.51


In [ ]:
# 출하지수(원지수) : Istans > 주제별 통계 > 산업동향지수 > 출하지수(원지수)
url = "https://istans.or.kr/su/newSuTab.do?scode=S351"
iSTANSCallResult = iSTANSCall(url)
dfSHIPMENT_INDEX = iSTANSCallResult.rename(columns = {'indexValue':'SHIPMENT_INDEX'})
dfSHIPMENT_INDEX = dfSHIPMENT_INDEX[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'SHIPMENT_INDEX']]
dfSHIPMENT_INDEX.drop_duplicates(inplace=True)
display(dfSHIPMENT_INDEX)

,STD_YM,EFAS_CD,EFAS_NM,SHIPMENT_INDEX
0,201708,14,의약,109.4
1,201707,14,의약,115.9
2,201101,14,의약,87.88
3,202107,14,의약,152.1
4,201010,14,의약,92.87
...,...,...,...,...
10753,201809,42,가구,89.6
10760,201311,42,가구,98.85
10770,202005,42,가구,103.7
10771,201509,42,가구,94.4


In [ ]:
# 출하지수(계절조정) : Istans > 주제별 통계 > 산업동향지수 > 출하지수(계절조정)
url = "https://istans.or.kr/su/newSuTab.do?scode=S352"
iSTANSCallResult = iSTANSCall(url)
dfSHIPMENT_INDEX_GAE = iSTANSCallResult.rename(columns = {'indexValue':'SHIPMENT_INDEX_GAE'})
dfSHIPMENT_INDEX_GAE = dfSHIPMENT_INDEX_GAE[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'SHIPMENT_INDEX_GAE']]
dfSHIPMENT_INDEX_GAE.drop_duplicates(inplace=True)
display(dfSHIPMENT_INDEX_GAE)

,STD_YM,EFAS_CD,EFAS_NM,SHIPMENT_INDEX_GAE
0,201708,14,의약,116.3
1,201707,14,의약,116.0
2,201101,14,의약,90.66
3,202107,14,의약,148.4
4,201010,14,의약,89.77
...,...,...,...,...
10753,201809,42,가구,100.9
10760,201311,42,가구,93.01
10770,202005,42,가구,106.7
10771,201509,42,가구,102.0


In [ ]:
# 재고지수(원지수) : Istans > 주제별 통계 > 산업동향지수 > 재고지수(원지수)
url = "https://istans.or.kr/su/newSuTab.do?scode=S353"
iSTANSCallResult = iSTANSCall(url)
dfINVENTORY_INDEX = iSTANSCallResult.rename(columns = {'indexValue':'INVENTORY_INDEX'})
dfINVENTORY_INDEX = dfINVENTORY_INDEX[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'INVENTORY_INDEX']]
dfINVENTORY_INDEX.drop_duplicates(inplace=True)
display(dfINVENTORY_INDEX)

,STD_YM,EFAS_CD,EFAS_NM,INVENTORY_INDEX
0,201708,14,의약,NaN
1,201707,14,의약,NaN
2,201101,14,의약,NaN
3,202107,14,의약,NaN
4,201010,14,의약,NaN
...,...,...,...,...
10753,201809,42,가구,151.3
10760,201311,42,가구,115.61
10770,202005,42,가구,132.4
10771,201509,42,가구,100.2


In [ ]:
# 재고지수(계절조정) : Istans > 주제별 통계 > 산업동향지수 > 재고지수(계절조정)
url = "https://istans.or.kr/su/newSuTab.do?scode=S354"
iSTANSCallResult = iSTANSCall(url)
dfINVENTORY_INDEX_GAE = iSTANSCallResult.rename(columns = {'indexValue':'INVENTORY_INDEX_GAE'})
dfINVENTORY_INDEX_GAE = dfINVENTORY_INDEX_GAE[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'INVENTORY_INDEX_GAE']]
dfINVENTORY_INDEX_GAE.drop_duplicates(inplace=True)
display(dfINVENTORY_INDEX_GAE)

,STD_YM,EFAS_CD,EFAS_NM,INVENTORY_INDEX_GAE
0,201708,14,의약,NaN
1,201707,14,의약,NaN
2,201101,14,의약,NaN
3,202107,14,의약,NaN
4,201010,14,의약,NaN
...,...,...,...,...
10753,201809,42,가구,152.1
10760,201311,42,가구,113.1
10770,202005,42,가구,128.6
10771,201509,42,가구,100.0


In [ ]:
# 가동률지수(원지수) : Istans > 주제별 통계 > 산업동향지수 > 가동률지수(원지수)
url = "https://istans.or.kr/su/newSuTab.do?scode=S355"
iSTANSCallResult = iSTANSCall(url)
dfOPER_RATE_INDEX = iSTANSCallResult.rename(columns = {'indexValue':'OPER_RATE_INDEX'})
dfOPER_RATE_INDEX = dfOPER_RATE_INDEX[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'OPER_RATE_INDEX']]
dfOPER_RATE_INDEX.drop_duplicates(inplace=True)
display(dfOPER_RATE_INDEX)

,STD_YM,EFAS_CD,EFAS_NM,OPER_RATE_INDEX
0,201708,14,의약,NaN
1,201707,14,의약,NaN
2,201101,14,의약,NaN
3,202107,14,의약,NaN
4,201010,14,의약,NaN
...,...,...,...,...
10753,201809,42,가구,104.4
10760,201311,42,가구,110.61
10770,202005,42,가구,96.3
10771,201509,42,가구,96.7


In [ ]:
# 가동률지수(계절조정) : Istans > 주제별 통계 > 산업동향지수 > 가동률지수(계절조정)
url = "https://istans.or.kr/su/newSuTab.do?scode=S356"
iSTANSCallResult = iSTANSCall(url)
dfOPER_RATE_INDEX_GAE = iSTANSCallResult.rename(columns = {'indexValue':'OPER_RATE_INDEX_GAE'})
dfOPER_RATE_INDEX_GAE = dfOPER_RATE_INDEX_GAE[['STD_YM', 'EFAS_CD', 'EFAS_NM', 'OPER_RATE_INDEX_GAE']]
dfOPER_RATE_INDEX_GAE.drop_duplicates(inplace=True)
display(dfOPER_RATE_INDEX_GAE)

,STD_YM,EFAS_CD,EFAS_NM,OPER_RATE_INDEX_GAE
0,201708,14,의약,NaN
1,201707,14,의약,NaN
2,201101,14,의약,NaN
3,202107,14,의약,NaN
4,201010,14,의약,NaN
...,...,...,...,...
10753,201809,42,가구,116.4
10760,201311,42,가구,102.17
10770,202005,42,가구,99.8
10771,201509,42,가구,103.5


In [ ]:
# compile the list of dataframes you want to merge
df_list = [dfPRODUC_INDEX, dfPRODUC_INDEX_GAE, dfSHIPMENT_INDEX, dfSHIPMENT_INDEX_GAE, dfINVENTORY_INDEX, dfINVENTORY_INDEX_GAE, dfOPER_RATE_INDEX, dfOPER_RATE_INDEX_GAE]
# grab first dataframe
IT_EFAS_INDU_COMMON_DB= df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    IT_EFAS_INDU_COMMON_DB = IT_EFAS_INDU_COMMON_DB.merge(to_merge, left_on=['STD_YM', 'EFAS_CD', 'EFAS_NM'], right_on=['STD_YM', 'EFAS_CD', 'EFAS_NM'], how="outer")
    
IT_EFAS_INDU_COMMON_DB.drop_duplicates(inplace=True)
IT_EFAS_INDU_COMMON_DB.set_index('STD_YM', inplace=True)
IT_EFAS_INDU_COMMON_DB['EFAS_CD'] = pd.to_numeric(IT_EFAS_INDU_COMMON_DB['EFAS_CD'])
IT_EFAS_INDU_COMMON_DB = IT_EFAS_INDU_COMMON_DB.sort_values(['STD_YM', 'EFAS_CD'], ascending = [True, True])
IT_EFAS_INDU_COMMON_DB['EFAS_CD'] = IT_EFAS_INDU_COMMON_DB['EFAS_CD'].astype(str)

today = date.today()
IT_EFAS_INDU_COMMON_DB.to_excel(f'{today}_IT_EFAS_INDU_COMMON_DB.xlsx')

IT_EFAS_INDU_COMMON_DB

,EFAS_CD,EFAS_NM,PRODUC_INDEX,PRODUC_INDEX_GAE,SHIPMENT_INDEX,SHIPMENT_INDEX_GAE,INVENTORY_INDEX,INVENTORY_INDEX_GAE,OPER_RATE_INDEX,OPER_RATE_INDEX_GAE
STD_YM,,,,,,,,,,
201001,3,음식료,90.74,89.38,89.58,89.17,70.03,71.76,99.98,96.86
201001,4,섬유,103.33,106.7,100.69,106.53,80.99,80.21,101.87,103.74
201001,5,의류,128.84,119.3,117.69,112.61,93.89,92.54,NaN,NaN
201001,6,가죽신발,99.35,97.33,99.49,98.94,80.47,81.31,109.82,107.37
201001,7,목재,98.27,105.62,94.92,104.48,99.8,97.91,88.95,95.18
...,...,...,...,...,...,...,...,...,...,...
202210,62,교육,90.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202210,63,의료·보건,157.51,156.36,NaN,NaN,NaN,NaN,NaN,NaN
202210,64,사회복지,119.29,119.11,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Add EXPORT_MDOLLAR and EMPORT_MDOLLAR table
dfEfasExportImort = pd.read_excel('IT_EFAS_ExportImort.xlsx')

IT_EFAS_INDU_COMMON_DB = IT_EFAS_INDU_COMMON_DB.merge(dfEfasExportImort, left_on=['STD_YM', 'EFAS_CD', 'EFAS_NM'], right_on=['STD_YM', 'EFAS_CD', 'EFAS_NM'], how="left")
IT_EFAS_INDU_COMMON_DB.drop_duplicates(inplace=True)
IT_EFAS_INDU_COMMON_DB.set_index('STD_YM', inplace=True)
IT_EFAS_INDU_COMMON_DB['EFAS_CD'] = pd.to_numeric(IT_EFAS_INDU_COMMON_DB['EFAS_CD'])
IT_EFAS_INDU_COMMON_DB = IT_EFAS_INDU_COMMON_DB.sort_values(['STD_YM', 'EFAS_CD'], ascending = [True, True])
IT_EFAS_INDU_COMMON_DB['EFAS_CD'] = IT_EFAS_INDU_COMMON_DB['EFAS_CD'].astype(str)